# Eval: 评估性能

## mem*.txt 文件处理  

In [1]:
# 文件格式  
# /media/Data/qipeng/modified_complete_images/AMNet-Rumor/AMNet-Predict/images/nonrumor/000e9fbgw1euz5ia6soqj20c811tjwf.jpg -0.0433418 0.0
# <[Path]+[ImgName]> <PredValue> <0.0>

# 处理结果: mem-expt*.txt
# <ImgName> <PredValue>

# 处理结果: mem-expt*-[threshold].txt
# <ImgName> <label>

# 处理结果: eval-expt*.txt
# <ImgName> <PredValue> <TrueLabel>

In [16]:
# 参数
expt_to_handle = 'expt1.2'
# 阈值
threshold = 0.5

In [17]:
import pandas as pd
import os

src_path = '../_expt/'
des_path = './'

path = '/media/Data/qipeng/modified_complete_images/AMNet-Rumor/AMNet-Predict/images/'

########################### mem-expt-sorted.txt 与 mem-expt-threshold-sorted.txt ###########################
mem_files = []
for root, dirs, files in os.walk(src_path, topdown=False):
    for file in files:
        if 'memorabilities' not in file:
            continue
        if expt_to_handle not in file:
            continue
            
        # 处理expt*.2
        if '.2' not in expt_to_handle and '.2' in file:
            continue
            
        mem_files.append(file)

print(mem_files)

with open(des_path + 'mem-' + expt_to_handle + '-sorted.txt', 'w') as out1:
    with open(des_path + 'mem-' + expt_to_handle + '-' + str(threshold) +'-sorted.txt', 'w') as out2:
        out1_lines = []
        out2_lines = []
        
        for mem_file in mem_files:
            with open(src_path + mem_file, 'r') as raw:
                lines = raw.readlines()
                img_dir = path + 'nonrumor/' if 'nonrumor' in mem_file else path + 'rumor/'
                for line in lines:
                    img_name, pred_value, _ = line.split()
                    img_name = img_name.split(img_dir)[1]
                    # out1.write(img_name + ' ' + pred_value + '\n')
                    out1_lines.append(img_name + ' ' + pred_value + '\n')
                    
                    label = 1 if float(pred_value) > threshold else 0
                    # out2.write(img_name + ' ' + str(label) + '\n')
                    out2_lines.append(img_name + ' ' + str(label) + '\n')
        
        out1_lines.sort()
        out2_lines.sort()
        out1.writelines(out1_lines)
        out2.writelines(out2_lines)

########################### eval-expt*.txt ###########################

eval_file = des_path + 'eval-' + expt_to_handle + '.csv'
true_file = 'pred-real-sorted.txt'
pred_file = des_path + 'mem-' + expt_to_handle + '-sorted.txt'

true_df = pd.read_csv(true_file, header=None, delim_whitespace=True)
pred_df = pd.read_csv(pred_file, header=None, delim_whitespace=True)
pred_df[2] = true_df[1]

pred_df.to_csv(eval_file, header=['ImgName', 'PredValue', 'Label'], index=None)

['memorabilities-expt1.2-nonrumor.txt', 'memorabilities-expt1.2-rumor.txt']


## Accuracy

In [24]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

real_file = 'pred-real-sorted.txt'
pred_file = 'mem-' + expt_to_handle + '-' + str(threshold) + '-sorted.txt'

real_df = pd.read_csv(real_file, header=None, delim_whitespace=True)
pred_df = pd.read_csv(pred_file, header=None, delim_whitespace=True)

y_real = real_df[1].tolist()
y_pred = pred_df[1].tolist()

In [25]:
metrics.accuracy_score(y_real, y_pred)

0.74814814814814812

In [26]:
print(metrics.classification_report(y_real, y_pred))

             precision    recall  f1-score   support

          0       0.71      0.83      0.77      2700
          1       0.80      0.67      0.73      2700

avg / total       0.76      0.75      0.75      5400



## AUC

In [27]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

real_file = 'pred-real-sorted.txt'
pred_file = 'mem-' + expt_to_handle + '-sorted.txt'

real_df = pd.read_csv(real_file, header=None, delim_whitespace=True)
pred_df = pd.read_csv(pred_file, header=None, delim_whitespace=True)

y_real = real_df[1].tolist()
y_pred = pred_df[1].tolist()

In [28]:
print(metrics.roc_auc_score(y_real, y_pred))

0.840358916324
